# Tree crown detection and multi-class classification (DeepForest)

## Context
### Purpose
Detect and classify tree crowns using a pre-trained Deep Learning model.

### Modelling Approach
The [CropModel](https://deepforest.readthedocs.io/en/latest/CropModels.html) from the latest version (v1.4.0) of the [DeepForest](https://deepforest.readthedocs.io/en/latest/) Deep Learning model is used to predict bounding boxes corresponding to individual tree crowns from airborn RGB images, and then perfom multi-class classifaction on each of those detections. 

DeepForest was trained on data from the National Ecological Observatory Network (NEON), and implemented using pytorch.

### Description
This notebook will explore the performance of DeepForest's pre-trained models on different publicly available pre-processed data:

- Homogeneous vs heterogeneous landscape
- Small distinct tree-crowns vs large dense canopies
- High vs low resolution images
- Tiles vs entire images

Additionally, it will cover post-processing steps to evaluate and visualise the model's predictions.

### Highlights
*Provide 3-5 bullet points that convey the use case’s core procedures. Each bullet point must have a maximum of 85 characters, including spaces.*
* Highlight 1
* Highlight 2

### Contributions
#### Notebook
* Cameron Appel (author), Queen Mary University of London, @camappel

#### Modelling codebase
* Ben Weinstein (maintainer & developer), University of Florida, @bw4sz
* Henry Senyondo (support maintainer), University of Florida, @henrykironde
* Ethan White (PI and author), University of Florida, @weecology


## Load libraries
List libraries according to their role e.g. system/files manipulation i.e. os (first), data handling i.e. numpy, xarray (second), visualisation i.e. holoviews (third), etc. The cell below contains two libraries, `os` and `warning` which are common among the Python Jupyter notebooks. Don't remove them.*

In [6]:
import os

import pooch

import warnings
warnings.filterwarnings(action='ignore')

ModuleNotFoundError: No module named 'pooch'

## Set project structure
*The cell below creates a separate folder to save the notebook outputs. This facilitates the reader to inspect inputs/outputs stored within a defined destination folder. Don't remove the lines below.*

In [ ]:
notebook_folder = './notebook'
if not os.path.exists(notebook_folder):
    os.makedirs(notebook_folder)

## Fetch RGB images from Zenodo

Fetch sample images from the publicly accessible [NEON](https://zenodo.org/records/3459803) training set to evaluate DeepForest's detection and classification performance on different types of landscapes

### Small distinct tree-crowns

In [ ]:
pooch.retrieve(
    url="doi:10.5281/zenodo.3459803/2018_SJER_3_258000_4106000_image.tif",
    # known_hash="md5:01a7cf23b368ff9e006fda8fe9ca4c8c",
    path=notebook_folder,
    fname="2018_SJER_3_258000_4106000_image.tif"
)

### Large dense canopies

In [ ]:
pooch.retrieve(
    # url="doi:10.5281/zenodo.3459803/2018_SJER_3_258000_4106000_image.tif",
    # known_hash="md5:01a7cf23b368ff9e006fda8fe9ca4c8c",
    path=notebook_folder,
    fname="2018_MLBS_3_541000_4140000_image_crop2.tif"
)

## Load data/model
*Load full dataset or model from original or mirror sources. If the license of the dataset/model permits, we suggest creating sample data (preprocessed) or model mirror for the notebook stored in a data repository e.g. Zenodo.*

In [ ]:
from deepforest.model import CropModel

model = CropModel()

## Visualisation
*Create a visual narrative of the dataset! We suggest exploring libraries suited for interactive plotting e.g. python (Holoviews, Panel, Bokeh).*

## Summary
*Provide 3-5 bullet points summarising the main aspects of the dataset and tools covered in the notebook.*

* Sentence 1 e.g. `tool-name` to perform...
* Sentence 2 e.g. `tool-name` to perform...

## Additional information
**Dataset**: Type here details of dataset(s) version.

**Codebase**: Type here details of codebase version (only for notebooks categorised under modelling/preprocesing/post-processing themes).

**License**: The code in this notebook is licensed under the MIT License. The Environmental Data Science book is licensed under the Creative Commons by Attribution 4.0 license. See further details [here](https://github.com/alan-turing-institute/environmental-ds-book/blob/master/LICENSE.md).

**Contact**: If you have any suggestion or report an issue with this notebook, feel free to [create an issue](https://github.com/alan-turing-institute/environmental-ds-book/issues/new/choose) or send a direct message to [environmental.ds.book@gmail.com](mailto:environmental.ds.book@gmail.com).

In [ ]:
from datetime import date
print(f'Last tested: {date.today()}')